In [1]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 46.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 116.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [39]:
!python src/train.py

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Mejores hiperparámetros encontrados: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Modelo entrenado y guardado exitosamente.
Error Absoluto Medio (MAE): 4.215644754316562
Raíz del Error Cuadrático Medio (RMSE): 5.380471563529822
Coeficiente de Determinación (R²): 0.46803856407515254
MAE promedio con validación cruzada: 4.152095791744229


In [30]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Productos base obligatorios
base_productos = [
    "Abrigo", "Botas de agua", "Bufanda", "Calcetines", "Camiseta", "Chandal", "Chaqueta",
    "Frutas", "Gafas de sol", "Gorra", "Guantes", "Impermeable", "Leche", "Pan", "Pantalon",
    "Pijama", "Sandalias", "Shorts", "Sombrilla", "Sudadera", "Traje de Agua", "Zapatos deportivos"
]

# Asignar meses a estaciones
estaciones_meses = {
    "Primavera": [9, 10, 11],
    "Verano": [12, 1, 2],
    "Otoño": [3, 4, 5],
    "Invierno": [6, 7, 8],
}

# Generar fechas aleatorias
fechas = list(pd.date_range(start="2020-01-01", end="2025-05-01", freq='D'))
np.random.shuffle(fechas)

# Función para generar una fila de datos
def generar_fila(fecha):
    producto = np.random.choice(base_productos)
    mes = fecha.month
    dia_semana = fecha.weekday()
    es_fin_semana = int(dia_semana >= 5)
    estacion = next(est for est, meses in estaciones_meses.items() if mes in meses)

    temp = {
        "Verano": np.random.normal(30, 5),
        "Primavera": np.random.normal(20, 4),
        "Otoño": np.random.normal(15, 4),
        "Invierno": np.random.normal(5, 5),
    }[estacion]

    lluvia = max(0, np.random.normal({
        "Verano": 0.5,
        "Primavera": 2,
        "Otoño": 3,
        "Invierno": 4
    }[estacion], 1.5))

    stock = max(1, int(np.random.normal(20, 10)))

    # Regla para calcular ventas basada en correlaciones
    ventas_base = stock * np.random.uniform(0.2, 0.8)
    if estacion == "Invierno" and producto in ["Abrigo", "Guantes", "Bufanda", "Impermeable", "Traje de Agua"]:
        ventas_base += np.random.uniform(5, 15)
    if estacion == "Verano" and producto in ["Gafas de sol", "Sombrilla", "Sandalias", "Shorts"]:
        ventas_base += np.random.uniform(5, 15)
    if producto in ["Sandalias", "Shorts"]:
        ventas_base -= lluvia * 0.5
    if producto in ["Botas de agua", "Impermeable"]:
        ventas_base += lluvia * 0.7
    if es_fin_semana and producto in ["Pan", "Leche", "Frutas"]:
        ventas_base += 5

    ventas = max(0, int(np.random.normal(ventas_base, 3)))

    return {
        "fecha": fecha.strftime("%Y-%m-%d"),
        "producto": producto,
        "stock": stock,
        "temp": round(temp, 2),
        "lluvia": round(lluvia, 2),
        "mes": mes,
        "dia_semana": dia_semana,
        "es_fin_semana": es_fin_semana,
        "estacion": estacion,
        "ventas": ventas
    }

# Generar dataset
datos = [generar_fila(f) for f in fechas[:15000]]
df = pd.DataFrame(datos)

# Guardar en CSV
df.to_csv("dataset_sintetico.csv", index=False)
print("CSV generado como 'dataset_sintetico.csv'")


CSV generado como 'dataset_sintetico.csv'
